In [1]:
from Data_processing import import_images as i
from Classification import classification_class as classification
import pandas as pd

=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
=========1. Extract Edge Detection features from all images=========
=========2. Convert feature list to DataFrame=========
=========3. Normalize features=========
   canny_edge_density  sobelx_mean  sobelx_std  sobely_mean  sobely_std  \
0            0.616994     0.564201    0.279468     0.590818    0.374775   
1            0.548750     0.677748    0.645522     0.398871    0.281901   
2            0.258691     0.769452    0.301940     0.493734    0.294019   
3            0.420246     0.576336    0.101811     0.548493    0.323417   
4            0.420778     0.502383    0.303101     0.537552    0.315156   

   prewitt_edge_density  laplacian_edge_density  roberts_edge_density  \
0              0.346193                0.714930     

In [2]:
metrics_results_best_methods = {
        "accuracy" : {},
        "f1-score" : {},
        "recall" : {},
        "precision" : {},
        "roc_auc" : {}
        }


name_best_models = {}

In [3]:
from Features_extraction import SIFT as SIFT
print("========= 5. Classification =========")
df_features = pd.DataFrame(SIFT.feature_vectors)
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["SIFT + BoW"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW", metrics_results_best_methods)

========= 5. Classification =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.80      0.78        86
           1       0.61      0.54      0.57        50

    accuracy                           0.71       136
   macro avg       0.68      0.67      0.67       136
weighted avg       0.70      0.71      0.70       136



In [4]:
metrics_results_best_methods

{'accuracy': {'SIFT + BoW': 0.7058823529411765},
 'f1-score': {'SIFT + BoW': 0.6748744919913938},
 'recall': {'SIFT + BoW': 0.6711627906976745},
 'precision': {'SIFT + BoW': 0.6818181818181819},
 'roc_auc': {'SIFT + BoW': 0.6711627906976745}}

In [5]:
from Features_extraction import edge_detection as edges
print("========= 4. Classification =========")
df_features_edge = edges.features_df
df_Y = pd.DataFrame(i.Y)
assert len(df_features_edge) == len(df_Y), "Mismatch entre features et labels !"



data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["EDGE"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE", metrics_results_best_methods)



========= 4. Classification =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.83      0.86        86
           1       0.74      0.84      0.79        50

    accuracy                           0.83       136
   macro avg       0.82      0.83      0.82       136
weighted avg       0.84      0.83      0.83       136



In [6]:
metrics_results_best_methods

{'accuracy': {'SIFT + BoW': 0.7058823529411765, 'EDGE': 0.8308823529411765},
 'f1-score': {'SIFT + BoW': 0.6748744919913938, 'EDGE': 0.8228263947890115},
 'recall': {'SIFT + BoW': 0.6711627906976745, 'EDGE': 0.8327906976744186},
 'precision': {'SIFT + BoW': 0.6818181818181819, 'EDGE': 0.8177881412391739},
 'roc_auc': {'SIFT + BoW': 0.6711627906976745, 'EDGE': 0.8327906976744185}}

# Comparison after preprocessing

In [9]:
from Data_processing import data_preprocessing as dp
import cv2
import os
import numpy as np


# Apply preprocessing to all images
print("=========  Applying Preprocessing to Images =========")

# Load images 
target_size = (128, 128)  # Mettre la taille désirée

preprocessed_images = [cv2.resize(img, target_size, interpolation=cv2.INTER_AREA) for img in preprocessed_images]
preprocessed_images = np.array(preprocessed_images)

# Convert back to NumPy array 
preprocessed_images = np.array(preprocessed_images)

# Update dataset with preprocessed images
i.images = preprocessed_images


=========  Applying Preprocessing to Images =========


In [11]:
from Features_extraction import edge_detection as edges

print("========= Extracting Edge Detection Features =========")

df_features_edge = edges.features_df
df_Y = pd.DataFrame(i.Y)

assert len(df_features_edge) == len(df_Y), "Mismatch between features and labels!"

data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name for Edge Detection: {best_method_name}")
name_best_models["EDGE (Preprocessed)"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE (Preprocessed)", metrics_results_best_methods)


========= Extracting Edge Detection Features =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name for Edge Detection: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.83      0.86        86
           1       0.74      0.84      0.79        50

    accuracy                           0.83       136
   macro avg       0.82      0.83      0.82       136
weighted avg       0.84      0.83      0.83       136



In [12]:
# Convert results to DataFrame for better visualization
df_results = pd.DataFrame(metrics_results_best_methods)
print("\n========= Comparison of Classification Results =========")
print(df_results)


========= Comparison of Classification Results =========
                     accuracy  f1-score    recall  precision   roc_auc
SIFT + BoW           0.705882  0.674874  0.671163   0.681818  0.671163
EDGE                 0.830882  0.822826  0.832791   0.817788  0.832791
EDGE (Preprocessed)  0.830882  0.822826  0.832791   0.817788  0.832791
